# Machine Learning - Final assignment

Mutaz Abueisheh</br>
Marcelo Jose Ferrer</br>
Maximiliano Hormazábal Lagos</br>
Mohamed Aymen Merchaoui</br>

## Classification problem

Classify the dataset between this 10 classes.

0 = Acoustic/Folk</br>
1 = Alternative music</br>
2 = Blues</br>
3 = Bollywood</br>
4 = Country</br>
5 = Hip Hop</br>
6 = Indie</br>
7 = Instrumental</br>
8 = Metal</br>
9 = Pop</br>
10 = Rock</br>

https://www.kaggle.com/datasets/purumalgi/music-genre-classification

In [1]:
using ScikitLearn

@sk_import svm:SVC
@sk_import tree:DecisionTreeClassifier
@sk_import linear_model:LogisticRegression
@sk_import naive_bayes:GaussianNB 
@sk_import ensemble:VotingClassifier
@sk_import ensemble:StackingClassifier
@sk_import ensemble:BaggingClassifier

PyObject <class 'sklearn.ensemble._bagging.BaggingClassifier'>

In [2]:
# Include the code done in previous practices
include("utils/practices_code.jl")
# Class that handle the data processing
include("utils/data_handler.jl")

modelCrossValidation (generic function with 1 method)

In [3]:
# Load the dataset and normalize
dataset = readdlm("dataset/music_genre.csv",',');

println("Dataset original size: ", size(dataset))
println("Sample of original dataset: ", dataset[2,1:17])

# Remove line of headers, artist and song name. Separate train_x and train_y
train_x = dataset[2:size(dataset,1),3:16]
train_y = dataset[2:size(dataset,1),17]

# Replace all empty values for zero
train_x = replace(train_x, "" => 0)

# TODO: Transform column duration in min/ms to only ms
# train_x[:12] = train_x[:12]

# Transform columns to positive values
train_x = abs.(train_x)

# Convert all inputs to real for simplification
train_x=convert(Array{Real,2},train_x); 

normalizedinputs = normalizeMinMax(train_x)

println("Inputs size: ", size(train_x))
println("Sample of inputs: ", train_x[1,1:14])
println("Outputs size: ", size(train_y))
println("Sample of Outputs: ", train_y[1])

Dataset original size: (17997, 17)
Sample of original dataset: Any["Bruno Mars", "That's What I Like (feat. Gucci Mane)", 60.0, 0.854, 0.564, 1.0, -4.964, 1, 0.0485, 0.0171, "", 0.0849, 0.899, 134.071, 234596.0, 4, 5]
Inputs size: (17996, 14)
Sample of inputs: Real[60.0, 0.854, 0.564, 1.0, 4.964, 1, 0.0485, 0.0171, 0, 0.0849, 0.899, 134.071, 234596.0, 4]
Outputs size: (17996,)
Sample of Outputs: 5


In [4]:
# Using Hold Out function to split dataset into train, test and validation
indexs = holdOut(size(train_x,1),0.2,0.1)

train_input = normalizedinputs[indexs[1],:]
train_output = train_y[indexs[1]]

test_input = normalizedinputs[indexs[2],:]
test_output = train_y[indexs[2]]

validation_input = normalizedinputs[indexs[3],:]
validation_output = train_y[indexs[3]]

println("Size original input data: ", size(train_x))
println("Size original output data: ", size(train_y))

println("Size train input data: ", size(train_input))
println("Size train output data: ", size(train_output))

println("Size test input data: ", size(test_input))
println("Size test output data: ", size(test_output))

println("Size validation input data: ", size(validation_input))
println("Size validation output data: ", size(validation_output))

Size original input data: (17996, 14)
Size original output data: (17996,)
Size train input data: (12597, 14)
Size train output data: (12597,)
Size test input data: (3599, 14)
Size test output data: (3599,)
Size validation input data: (1800, 14)
Size validation output data: (1800,)


In [ ]:
#Define the models to train
models = Dict( "SVM" => SVC(probability=true), 
         "LR" =>LogisticRegression(),
         "DT"=> DecisionTreeClassifier(max_depth=4),
         "NB"=> GaussianNB())

base_models =  [ name for name in keys(models)]

In [ ]:
# Perform the training for each model and calculate the test values (accuracy)
for key in keys(models)
    model = models[key]
    fit!(model,train_input, train_output)
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

In [ ]:
#Define the metaclassifier based on the base_models
models["Ensemble (Hard Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1)
fit!(models["Ensemble (Hard Voting)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

In [ ]:
models["Ensemble (Soft Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1, voting="soft",weights=[1,2,2,1])
fit!(models["Ensemble (Soft Voting)"],train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end

In [ ]:
models["Ensemble (Stacking)"] = StackingClassifier(estimators=[(name,models[name]) for name in base_models],
    final_estimator=SVC(probability=true), n_jobs=-1)
fit!(models["Ensemble (Stacking)"], train_input, train_output)

In [ ]:
for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end

In [ ]:
models["Bagging (SVC)"] = BaggingClassifier(base_estimator=SVC(),n_estimators=10, max_samples=0.50, n_jobs=-1)
fit!(models["Bagging (SVC)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end

In [ ]:
@sk_import ensemble:(AdaBoostClassifier, GradientBoostingClassifier)

models["Ada"] = AdaBoostClassifier(n_estimators=30)
fit!(models["Ada"], train_input, train_output)

models["GTB"] = GradientBoostingClassifier(n_estimators=30, learning_rate=1.0, max_depth=2, random_state=0)
fit!(models["GTB"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end


In [ ]:
@sk_import ensemble:RandomForestClassifier

models["RF"] = RandomForestClassifier(n_estimators=8, max_depth=nothing,
                                    min_samples_split=2, n_jobs=-1)
fit!(models["RF"], train_input, train_output)
    
for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

In [ ]:
p = bar(y=1:60,models["RF"].feature_importances_, orientation=:horizontal, legend = false)
xlabel!(p,"Gini Gain")
ylabel!(p,"Fearure")
title!("Feature Importance")

In [ ]:
using Pkg;
Pkg.add("XGBoost")

In [ ]:
using XGBoost;

train_output_asNumber= Vector{Number}(train_output);

@assert train_output_asNumber isa Vector{Number}

In [ ]:
model = xgboost(train_input, 20, label = train_output_asNumber, eta = 1, max_depth = 6)

In [ ]:
param = ["max_depth" => 2,
         "eta" => 1,
         "objective" => "binary:logistic"]
metrics = metrics = ["error", "auc"]
model = xgboost(train_input, 20, label = train_output_asNumber, param = param, metrics = metrics)

pred = predict(model, train_input)

In [ ]:
using XGBoost: predict as predict_xgb

pred = predict_xgb(model, test_input)
print("Error of XGboost= ", sum((pred .> 0.5) .!= test_output) / float(size(pred)[1]), "\n")

In [ ]:
feature_gain = map(x-> (x.fname,x.gain), importance(model))
feature, gain = first.(feature_gain), last.(feature_gain)

using Plots;

p = bar(gain, y=feature, orientation="h", legend=false)
xlabel!(p,"Gain")
ylabel!(p,"Feature")
title!("Feature Importance")